# AI!

In [ ]:
```python
import nbformat
from nbformat.v4 import new_notebook, new_code_cell
from pathlib import Path
from nbmake import NotebookRun

# Step 1: Create a hello world notebook using nbformat
nb = new_notebook()
code = "print('Hello, World!')"
nb.cells.append(new_code_cell(code))
nbformat.write(nb, 'x.ipynb')

# Step 2: Use nbmake's NotebookRun class to execute it from a Python application
notebook_run = NotebookRun(filename=Path('x.ipynb'), default_timeout=60)
result = notebook_run.execute()

# Step 3: Check the output notebook printed what we were expecting
assert result.nb['cells'][0]['outputs'][0]['text'] == 'Hello, World!\n'
```